In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,9,10,11,12,13,...,48,49,54,55,56,57,59,60,62,63
2,31.71,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,-0.0,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
3,-0.00,7.48,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00
4,-0.00,0.00,36.83,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,...,0.00,0.00,-0.0,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
5,0.00,0.00,-0.00,26.65,-0.00,0.00,0.00,-0.00,-0.00,0.00,...,0.00,-0.00,-0.0,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00
6,-0.00,0.00,0.00,-0.00,14.45,-0.00,-0.00,0.00,-0.00,-0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00
9,0.00,-0.00,0.00,0.00,-0.00,4.05,0.00,-0.00,-0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00
10,-0.00,0.00,0.00,0.00,-0.00,0.00,9.32,0.00,0.00,-0.00,...,-0.00,0.00,0.0,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
11,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,13.38,-0.00,0.00,...,0.00,-0.00,-0.0,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00
12,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,9.25,0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
13,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,11.09,...,0.00,0.00,-0.0,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,9,10,11,12,13,...,48,49,54,55,56,57,59,60,62,63
2,1.00000,-0.00001,-0.00002,0.00005,-0.00005,0.00002,-0.00002,0.00000,0.00002,-0.00003,...,-0.00004,0.00002,-0.00004,0.00003,-0.00005,-0.00000,-0.00000,-0.00004,0.00005,0.00003
3,-0.00001,1.00000,0.00002,0.00002,0.00002,-0.00002,0.00000,-0.00002,0.00001,-0.00003,...,-0.00002,-0.00000,0.00002,-0.00001,-0.00004,0.00002,0.00002,0.00003,0.00001,-0.00004
4,-0.00002,0.00002,1.00000,-0.00004,0.00000,0.00002,0.00002,-0.00006,-0.00003,0.00000,...,0.00002,0.00002,-0.00002,0.00004,0.00000,-0.00002,0.00003,-0.00002,0.00001,-0.00002
5,0.00005,0.00002,-0.00004,1.00000,-0.00002,0.00003,0.00004,-0.00000,-0.00001,0.00005,...,0.00002,-0.00003,-0.00003,-0.00002,-0.00006,0.00005,-0.00002,0.00000,0.00001,0.00004
6,-0.00005,0.00002,0.00000,-0.00002,1.00000,-0.00003,-0.00002,0.00003,-0.00005,-0.00002,...,-0.00009,-0.00001,0.00002,-0.00001,-0.00001,-0.00002,-0.00001,0.00003,-0.00002,0.00005
9,0.00002,-0.00002,0.00002,0.00003,-0.00003,1.00000,0.00003,-0.00003,-0.00005,0.00001,...,0.00001,0.00003,0.00007,0.00001,0.00006,0.00006,-0.00000,-0.00002,0.00002,-0.00001
10,-0.00002,0.00000,0.00002,0.00004,-0.00002,0.00003,1.00000,0.00001,0.00002,-0.00001,...,-0.00002,0.00005,0.00006,-0.00002,-0.00003,-0.00001,0.00005,0.00003,-0.00002,0.00000
11,0.00000,-0.00002,-0.00006,-0.00000,0.00003,-0.00003,0.00001,1.00000,-0.00003,0.00005,...,0.00002,-0.00001,-0.00000,-0.00003,0.00002,0.00000,0.00000,0.00002,-0.00003,0.00001
12,0.00002,0.00001,-0.00003,-0.00001,-0.00005,-0.00005,0.00002,-0.00003,1.00000,0.00002,...,-0.00000,-0.00000,0.00004,-0.00004,-0.00008,-0.00001,-0.00002,-0.00002,-0.00001,-0.00001
13,-0.00003,-0.00003,0.00000,0.00005,-0.00002,0.00001,-0.00001,0.00005,0.00002,1.00000,...,0.00000,0.00002,-0.00003,-0.00002,-0.00003,0.00004,0.00003,-0.00003,0.00005,0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.0166597382873403

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.91967665e+01 3.42766897e+01 2.89777287e+01 2.47310064e+01
 2.33866956e+01 2.14847164e+01 2.00813871e+01 1.72751412e+01
 1.59153465e+01 1.36764584e+01 1.29418594e+01 1.21832241e+01
 1.05714147e+01 5.95827482e+00 5.06024357e+00 3.56566778e+00
 3.34076593e+00 2.89361533e+00 2.77374382e+00 2.66512317e+00
 2.53587042e+00 2.35728644e+00 2.04664785e+00 1.64151299e+00
 1.59877865e+00 1.53015825e+00 1.46287133e+00 1.14350760e+00
 8.66111492e-01 6.56457560e-01 5.53783062e-01 4.98232542e-01
 3.72225989e-01 8.50975101e-02 1.07482704e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,-0.061788,-0.003712,0.983023,-0.027380,-0.008177,-0.001955,-0.004746,-0.007445,-0.004739,-0.005832,...,-0.002900,-0.019528,-0.001643,-0.005636,-0.034927,-0.005167,-0.029305,-0.000538,-0.013816,-0.002169
1,-0.137459,-0.004705,0.132925,-0.040549,-0.010440,-0.002436,-0.006000,-0.009399,-0.005937,-0.007420,...,-0.003644,-0.027154,-0.002035,-0.007177,-0.055959,-0.006511,-0.044504,-0.000675,-0.018398,-0.002719
2,0.971133,-0.004946,0.060988,-0.059193,-0.011468,-0.002558,-0.006383,-0.010190,-0.006290,-0.007930,...,-0.003850,-0.034277,-0.002153,-0.007620,-0.102197,-0.006923,-0.068869,-0.000708,-0.021224,-0.002826
3,0.067692,-0.003374,0.025086,-0.071031,-0.008055,-0.001743,-0.004378,-0.007111,-0.004250,-0.005466,...,-0.002609,-0.029781,-0.001432,-0.005262,-0.362606,-0.004792,-0.097407,-0.000464,-0.016306,-0.001934
4,0.084857,-0.005703,0.036674,-0.176142,-0.013799,-0.002891,-0.007411,-0.012127,-0.007339,-0.009293,...,-0.004388,-0.056484,-0.002435,-0.008911,0.882232,-0.008053,-0.289530,-0.000794,-0.028864,-0.003227
5,0.043398,-0.004155,0.021877,-0.496250,-0.010367,-0.002128,-0.005400,-0.009075,-0.005386,-0.006847,...,-0.003213,-0.052013,-0.001775,-0.006602,0.128605,-0.005960,0.845471,-0.000580,-0.023247,-0.002363
6,0.083093,-0.010094,0.046317,0.816869,-0.025839,-0.005127,-0.013193,-0.022485,-0.013140,-0.016737,...,-0.007760,-0.165189,-0.004302,-0.016197,0.185657,-0.014467,0.386305,-0.001396,-0.061432,-0.005683
7,0.034110,-0.006342,0.021424,0.097705,-0.017152,-0.003190,-0.008427,-0.014653,-0.008224,-0.010749,...,-0.004838,-0.407890,-0.002681,-0.010325,0.056820,-0.009191,0.078776,-0.000861,-0.049925,-0.003522
8,0.050541,-0.011428,0.033334,0.116749,-0.032200,-0.005692,-0.015145,-0.027333,-0.015007,-0.019627,...,-0.008702,0.863950,-0.004748,-0.018978,0.077448,-0.016726,0.099739,-0.001528,-0.110235,-0.006342
9,0.017127,-0.005376,0.012054,0.031837,-0.016540,-0.002639,-0.007214,-0.013732,-0.007173,-0.009494,...,-0.004018,0.071992,-0.002201,-0.009120,0.023764,-0.007959,0.028543,-0.000722,-0.100215,-0.002948


In [15]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([1.06178775, 1.00470477, 0.93901217, 1.07103053, 1.01379925,
       1.00212791, 1.01319295, 1.01465317, 1.01500709, 1.00949383,
       1.00784069, 1.10636312, 1.03614969, 0.98546406, 1.0282542 ,
       1.05339168, 1.01274838, 1.00780828, 1.01656628, 1.03522445,
       1.00241334, 1.1768738 , 0.98069604, 0.99923657, 1.00534587,
       0.96367041, 1.00403455, 0.94249021, 0.92985577, 0.98995204,
       0.97915705, 1.00381395, 0.9718087 , 1.04605201, 0.80552297])